In [1]:
import torch
from safetensors.torch import load_file

In [2]:
ct_movq_dict = dict(torch.load("/home/mlfavorfit/.cache/huggingface/hub/models--kandinsky-community--kandinsky-2-2-controlnet-depth/snapshots/4ecd717e8c9086cf4a16ca28b64894f70a42cd08/movq/diffusion_pytorch_model.bin"))
ct_unet_dict = dict(torch.load("/home/mlfavorfit/.cache/huggingface/hub/models--kandinsky-community--kandinsky-2-2-controlnet-depth/snapshots/4ecd717e8c9086cf4a16ca28b64894f70a42cd08/unet/diffusion_pytorch_model.bin"))

un_movq_dict = dict(load_file("/home/mlfavorfit/.cache/huggingface/hub/models--kandinsky-community--kandinsky-2-2-decoder-inpaint/snapshots/db790ad5cbcabed886f069ef2710774657621702/movq/diffusion_pytorch_model.safetensors"))
un_unet_dict = dict(load_file("/home/mlfavorfit/.cache/huggingface/hub/models--kandinsky-community--kandinsky-2-2-decoder-inpaint/snapshots/db790ad5cbcabed886f069ef2710774657621702/unet/diffusion_pytorch_model.safetensors"))

In [3]:
ct_movq = sorted([(cur, ct_movq_dict[cur].shape)for cur in ct_movq_dict], key=lambda x:x[0])
ct_unet = sorted([(cur, ct_unet_dict[cur].shape)for cur in ct_unet_dict], key=lambda x:x[0])
un_movq = sorted([(cur, un_movq_dict[cur].shape)for cur in un_movq_dict], key=lambda x:x[0])
un_unet = sorted([(cur, un_unet_dict[cur].shape)for cur in un_unet_dict], key=lambda x:x[0])

In [4]:
for ct, un in zip(ct_movq, un_movq):
    if ct != un: print("warn")

In [5]:
only_ct = []
common_ct_un = []
for name, shape in ct_unet:
    if name in un_unet_dict.keys():
        common_ct_un.append(name)
    else:
        only_ct.append(name)

In [6]:
for name in common_ct_un:
    if un_unet_dict[name].shape != ct_unet_dict[name].shape: print(name)

conv_in.weight


In [7]:
un_unet_dict["conv_in.weight"].shape

torch.Size([384, 9, 3, 3])

In [8]:
ct_unet_dict["conv_in.weight"].shape

torch.Size([384, 8, 3, 3])

In [9]:
for cur in only_ct:
    print(cur, ct_unet_dict[cur].shape)

add_embedding.input_hint_block.0.bias torch.Size([16])
add_embedding.input_hint_block.0.weight torch.Size([16, 3, 3, 3])
add_embedding.input_hint_block.10.bias torch.Size([96])
add_embedding.input_hint_block.10.weight torch.Size([96, 96, 3, 3])
add_embedding.input_hint_block.12.bias torch.Size([256])
add_embedding.input_hint_block.12.weight torch.Size([256, 96, 3, 3])
add_embedding.input_hint_block.14.bias torch.Size([4])
add_embedding.input_hint_block.14.weight torch.Size([4, 256, 3, 3])
add_embedding.input_hint_block.2.bias torch.Size([16])
add_embedding.input_hint_block.2.weight torch.Size([16, 16, 3, 3])
add_embedding.input_hint_block.4.bias torch.Size([32])
add_embedding.input_hint_block.4.weight torch.Size([32, 16, 3, 3])
add_embedding.input_hint_block.6.bias torch.Size([32])
add_embedding.input_hint_block.6.weight torch.Size([32, 32, 3, 3])
add_embedding.input_hint_block.8.bias torch.Size([96])
add_embedding.input_hint_block.8.weight torch.Size([96, 32, 3, 3])


In [10]:
import torch

# 기존 텐서의 모양과 새로운 모양 정의
old_shape = (384, 9, 3, 3)
new_shape = (384, 13, 3, 3)

# 새로운 텐서 생성 및 기존 텐서 값 복사
new_weight = torch.zeros(new_shape)  # 새로운 텐서를 모두 0으로 초기화
old_weight = un_unet_dict["conv_in.weight"]
new_weight[:, :old_shape[1], :, :] = old_weight

# 추가된 부분에 랜덤한 0으로 채우기
random_part = torch.zeros((new_shape[0], new_shape[1] - old_shape[1], new_shape[2], new_shape[3]))
new_weight[:, old_shape[1]:, :, :] = random_part

# 결과 확인
print(new_weight.shape)


torch.Size([384, 13, 3, 3])


In [11]:
un_unet_dict["conv_in.weight"] = new_weight

In [12]:
for cur in only_ct:
    un_unet_dict[cur] = ct_unet_dict[cur]

In [13]:
print(len(un_unet_dict))
print(un_unet_dict["conv_in.weight"].shape)

740
torch.Size([384, 13, 3, 3])


In [15]:
from safetensors.torch import save_file

save_file(un_unet_dict, "/home/mlfavorfit/Desktop/lib_link/favorfit/kjg/0_model_weights/diffusion/Kandinsky/kandinsky-inpainting-controlnet/unet/diffusion_pytorch_model.safetensors")

In [15]:
temp_dict = load_file("/home/mlfavorfit/Desktop/lib_link/favorfit/kjg/0_model_weights/diffusion/Kandinsky/kandinsky-inpainting-controlnet/unet/diffusion_pytorch_model.safetensors")

In [16]:
print(len(temp_dict))
print(temp_dict["conv_in.weight"].shape)

740
torch.Size([384, 13, 3, 3])


In [1]:
from PIL import Image

def downgrade_image(img, bad_size=[64,64]):
    img = img.resize(bad_size).resize(img.size)
    return img

from utils import make_canny_condition

In [6]:
from glob import glob
from PIL import Image

fns = glob("/media/mlfavorfit/sdb/contolnet_dataset/control_net_train_bad/images/*")

In [8]:
import os
from tqdm import tqdm

save_dir = "/media/mlfavorfit/sdb/contolnet_dataset/control_net_train_bad/conditioning_images_canny"
for fn in tqdm(fns, total=len(fns)):
    img = Image.open(fn)
    bad_img = make_canny_condition(img)
    bad_img.save(os.path.join(save_dir, os.path.basename(fn)))

100%|██████████| 197967/197967 [46:24<00:00, 71.10it/s] 


In [28]:
import torch
temp_key = torch.load("/home/mlfavorfit/Desktop/lib_link/favorfit/kjg/0_model_weights/diffusion/Kandinsky/kandinsky-decoder/controlnet/ckpt_63000_0313/diffusion_pytorch_model.bin", map_location="cpu").keys()
temp_key = list(temp_key)

In [29]:
len(temp_key)

392

In [30]:
from safetensors.torch import load_file

key_list = list(load_file("/home/mlfavorfit/Desktop/lib_link/favorfit/kjg/0_model_weights/diffusion/Kandinsky/kandinsky-decoder/unet/diffusion_pytorch_model.safetensors").keys())

In [33]:
len([cur for cur in key_list if not cur.startswith("up")])

334

In [46]:
temp = 0
for key in key_list:
    if key in temp_key:
        temp+=1
        if "mid" in key:
            print(key)
temp

mid_block.resnets.0.conv1.bias
mid_block.resnets.0.conv1.weight
mid_block.resnets.0.conv2.bias
mid_block.resnets.0.conv2.weight
mid_block.resnets.0.norm1.bias
mid_block.resnets.0.norm1.weight
mid_block.resnets.0.norm2.bias
mid_block.resnets.0.norm2.weight
mid_block.resnets.0.time_emb_proj.bias
mid_block.resnets.0.time_emb_proj.weight
mid_block.resnets.1.conv1.bias
mid_block.resnets.1.conv1.weight
mid_block.resnets.1.conv2.bias
mid_block.resnets.1.conv2.weight
mid_block.resnets.1.norm1.bias
mid_block.resnets.1.norm1.weight
mid_block.resnets.1.norm2.bias
mid_block.resnets.1.norm2.weight
mid_block.resnets.1.time_emb_proj.bias
mid_block.resnets.1.time_emb_proj.weight


316

In [43]:
temp = 0
for key in temp_key:
    if key not in key_list:
        print(key)
        temp+=1
temp

controlnet_cond_embedding.conv_in.weight
controlnet_cond_embedding.conv_in.bias
controlnet_cond_embedding.blocks.0.weight
controlnet_cond_embedding.blocks.0.bias
controlnet_cond_embedding.blocks.1.weight
controlnet_cond_embedding.blocks.1.bias
controlnet_cond_embedding.blocks.2.weight
controlnet_cond_embedding.blocks.2.bias
controlnet_cond_embedding.blocks.3.weight
controlnet_cond_embedding.blocks.3.bias
controlnet_cond_embedding.blocks.4.weight
controlnet_cond_embedding.blocks.4.bias
controlnet_cond_embedding.blocks.5.weight
controlnet_cond_embedding.blocks.5.bias
controlnet_cond_embedding.conv_out.weight
controlnet_cond_embedding.conv_out.bias
controlnet_down_blocks.0.weight
controlnet_down_blocks.0.bias
controlnet_down_blocks.1.weight
controlnet_down_blocks.1.bias
controlnet_down_blocks.2.weight
controlnet_down_blocks.2.bias
controlnet_down_blocks.3.weight
controlnet_down_blocks.3.bias
controlnet_down_blocks.4.weight
controlnet_down_blocks.4.bias
controlnet_down_blocks.5.weight
cont

76

In [42]:
temp = 0
for key in key_list:
    if key not in temp_key:
        temp+=1
temp

408

In [27]:
len([cur for cur in key_list if not cur.startswith("up")]) + 50

384

In [1]:
import random
from PIL import Image
from PIL.ImageOps import expand

from diffusers.utils import make_image_grid
from utils import *

import torch
from diffusers import KandinskyV22PriorPipeline
from diffusers.pipelines.kandinsky2_2.pipline_kandinsky2_2_controlnet_split import KandinskyV22ControlnetPipeline

from diffusers.models.controlnet_kandinsky import ControlNetModel as ControlNetModelKandinsky


/home/mlfavorfit/anaconda3/envs/diffusion_rnd/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
decoder_model_path = "/home/mlfavorfit/Desktop/lib_link/favorfit/kjg/0_model_weights/diffusion/Kandinsky/kandinsky-decoder"
prior_model_path = "/home/mlfavorfit/Desktop/lib_link/favorfit/kjg/0_model_weights/diffusion/Kandinsky/kandinsky-prior"

controlnet_kandinsky = ControlNetModelKandinsky.from_pretrained(
    decoder_model_path, subfolder="controlnet/tile_from_sd15", torch_dtype=torch.float16,
).eval()

# prior_pipeline = KandinskyV22PriorPipeline.from_pretrained(
#     prior_model_path,
#     use_safetensors=True,
#     torch_dtype=torch.float16,
# )

# pipeline = KandinskyV22ControlnetPipeline.from_pretrained(
#     decoder_model_path,
#     controlnet=controlnet_kandinsky,
#     torch_dtype=torch.float16,
# )

The config attributes {'center_input_sample': False} were passed to ControlNetModel, but are not expected and will be ignored. Please verify your config.json configuration file.


ValueError: Cannot load <class 'diffusers.models.controlnet_kandinsky.ControlNetModel'> from /home/mlfavorfit/Desktop/lib_link/favorfit/kjg/0_model_weights/diffusion/Kandinsky/kandinsky-decoder because the following keys are missing: 
 down_blocks.1.resnets.2.norm1.weight, down_blocks.0.attentions.2.transformer_blocks.0.attn2.to_out.0.weight, down_blocks.0.attentions.2.transformer_blocks.0.attn2.to_k.weight, down_blocks.1.attentions.2.transformer_blocks.0.ff.net.0.proj.weight, down_blocks.1.attentions.2.transformer_blocks.0.attn1.to_q.weight, down_blocks.1.attentions.2.transformer_blocks.0.norm3.weight, down_blocks.1.attentions.2.transformer_blocks.0.attn1.to_out.0.bias, down_blocks.1.attentions.2.transformer_blocks.0.attn1.to_out.0.weight, down_blocks.0.attentions.2.transformer_blocks.0.attn2.to_q.weight, down_blocks.0.attentions.2.transformer_blocks.0.ff.net.0.proj.bias, down_blocks.1.attentions.2.transformer_blocks.0.attn2.to_k.weight, down_blocks.2.attentions.2.transformer_blocks.0

In [2]:
import torch

wei = ControlNetModelKandinsky.from_config("/home/mlfavorfit/Desktop/lib_link/favorfit/kjg/0_model_weights/diffusion/Kandinsky/kandinsky-decoder/controlnet/tile_from_sd15/config.json").state_dict()
w = torch.load("/home/mlfavorfit/Desktop/lib_link/favorfit/kjg/0_model_weights/diffusion/Kandinsky/kandinsky-decoder/controlnet/tile_from_sd15/diffusion_pytorch_model.bin")

/home/mlfavorfit/lib/favorfit/kjg/diffusers_research/diffusers_research/diffusers/configuration_utils.py:244: FutureWarning: It is deprecated to pass a pretrained model name or path to `from_config`.If you were trying to load a model, please use <class 'diffusers.models.controlnet_kandinsky.ControlNetModel'>.load_config(...) followed by <class 'diffusers.models.controlnet_kandinsky.ControlNetModel'>.from_config(...) instead. Otherwise, please make sure to pass a configuration dictionary instead. This functionality will be removed in v1.0.0.
  deprecate("config-passed-as-path", "1.0.0", deprecation_message, standard_warn=False)
The config attributes {'center_input_sample': False} were passed to ControlNetModel, but are not expected and will be ignored. Please verify your config.json configuration file.


In [3]:
mu = 0.0
sigma = 0.2

for cur in wei.keys():
    if cur in w.keys():
        if wei[cur].shape != w[cur].shape: 
            print(cur)
            if cur.startswith("controlnet_down_blocks") or cur.startswith("controlnet_mid_block"):
                wei[cur] = torch.zeros_like(wei[cur])
            else:
                wei[cur] = sigma * torch.randn_like(wei[cur], dtype=torch.float16)
        else:
            wei[cur] = w[cur]
    else:
        print(cur)
        wei[cur] = sigma * torch.randn_like(wei[cur], dtype=torch.float16)
        if cur.startswith("controlnet_down_blocks") or cur.startswith("controlnet_mid_block"):
            wei[cur] = torch.zeros_like(wei[cur])

down_blocks.0.attentions.2.norm.weight
down_blocks.0.attentions.2.norm.bias
down_blocks.0.attentions.2.proj_in.weight
down_blocks.0.attentions.2.proj_in.bias
down_blocks.0.attentions.2.transformer_blocks.0.norm1.weight
down_blocks.0.attentions.2.transformer_blocks.0.norm1.bias
down_blocks.0.attentions.2.transformer_blocks.0.attn1.to_q.weight
down_blocks.0.attentions.2.transformer_blocks.0.attn1.to_k.weight
down_blocks.0.attentions.2.transformer_blocks.0.attn1.to_v.weight
down_blocks.0.attentions.2.transformer_blocks.0.attn1.to_out.0.weight
down_blocks.0.attentions.2.transformer_blocks.0.attn1.to_out.0.bias
down_blocks.0.attentions.2.transformer_blocks.0.norm2.weight
down_blocks.0.attentions.2.transformer_blocks.0.norm2.bias
down_blocks.0.attentions.2.transformer_blocks.0.attn2.to_q.weight
down_blocks.0.attentions.2.transformer_blocks.0.attn2.to_k.weight
down_blocks.0.attentions.2.transformer_blocks.0.attn2.to_v.weight
down_blocks.0.attentions.2.transformer_blocks.0.attn2.to_out.0.weigh

In [4]:
torch.min(wei["down_blocks.2.attentions.2.transformer_blocks.0.ff.net.2.weight"])

tensor(-0.3904, dtype=torch.float16)

In [6]:
wei["controlnet_mid_block.bias"]

tensor([0., 0., 0.,  ..., 0., 0., 0.])

In [7]:
wei["controlnet_proj_blocks.12.weight"]

tensor([[[[-0.1699,  0.2378, -0.0591],
          [ 0.2622, -0.4448,  0.3770],
          [-0.3022,  0.3354, -0.2798]],

         [[-0.2104,  0.2266, -0.0986],
          [-0.1406,  0.0220,  0.3018],
          [-0.4321, -0.0869, -0.1113]],

         [[-0.3682,  0.0581, -0.2690],
          [ 0.3193,  0.0918,  0.4707],
          [-0.1680,  0.1436,  0.4478]],

         ...,

         [[ 0.0620,  0.2100, -0.1523],
          [ 0.3569, -0.0557,  0.4468],
          [ 0.4136,  0.0400, -0.3149]],

         [[-0.3472,  0.3955,  0.4028],
          [ 0.3403, -0.1479, -0.3335],
          [ 0.2852, -0.3320, -0.2822]],

         [[ 0.2588,  0.4580, -0.0786],
          [ 0.3955,  0.2012,  0.0576],
          [-0.2373,  0.1382, -0.3965]]],


        [[[ 0.3003,  0.4888,  0.0361],
          [-0.1621, -0.4673,  0.4399],
          [ 0.4678, -0.4048,  0.0020]],

         [[ 0.1050,  0.3672,  0.3799],
          [-0.2139, -0.0591, -0.1265],
          [-0.1812,  0.0527, -0.1758]],

         [[ 0.2573,  0.3120, -0

In [13]:
torch.save(wei, "/home/mlfavorfit/Desktop/lib_link/favorfit/kjg/0_model_weights/diffusion/Kandinsky/kandinsky-decoder/controlnet/tile_from_sd15/diffusion_pytorch_model.bin")

In [31]:
with open("./temp.jsonl", mode="r") as f:
    temp = [cur[:-1] for cur in f]

In [32]:
temp

['down_blocks.0.attentions.2.norm.weight',
 'down_blocks.0.attentions.2.norm.bias',
 'down_blocks.0.attentions.2.proj_in.weight',
 'down_blocks.0.attentions.2.proj_in.bias',
 'down_blocks.0.attentions.2.transformer_blocks.0.norm1.weight',
 'down_blocks.0.attentions.2.transformer_blocks.0.norm1.bias',
 'down_blocks.0.attentions.2.transformer_blocks.0.attn1.to_q.weight',
 'down_blocks.0.attentions.2.transformer_blocks.0.attn1.to_k.weight',
 'down_blocks.0.attentions.2.transformer_blocks.0.attn1.to_v.weight',
 'down_blocks.0.attentions.2.transformer_blocks.0.attn1.to_out.0.weight',
 'down_blocks.0.attentions.2.transformer_blocks.0.attn1.to_out.0.bias',
 'down_blocks.0.attentions.2.transformer_blocks.0.norm2.weight',
 'down_blocks.0.attentions.2.transformer_blocks.0.norm2.bias',
 'down_blocks.0.attentions.2.transformer_blocks.0.attn2.to_q.weight',
 'down_blocks.0.attentions.2.transformer_blocks.0.attn2.to_k.weight',
 'down_blocks.0.attentions.2.transformer_blocks.0.attn2.to_v.weight',
 'do